---

# CS 369 2020 


Student: Rui Huang

UPI: rhua966


## Assignment 2 : Pairwise Sequence Alignment (7.5 points)

Due date: Tue 28/4/2020 at 23:59

 

### Instructions

This notebook contains automarking metadata, please
- __Update your name and UPI above;__
- __Do not change the file name;__
- __Do not change any cell starting with MANUAL MARKING and AUTOMARKING;__
- __Do not change any function signature, and do not change the data type of the function output.__ Moreover,  if it is specifically said that the function should return a specific data structure with specific names of columns like in Q1.iv) then your function should do that. Otherwise, autograding will give you zero points as the assert commands will fail when it will try to access the columns with expected names in the hidden test.
- __Before you turn this assignment in, make sure everything runs as expected.__ First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). A special note for the task Q1.iv) you can test the functions while working on the solution, but for the final submission, please comment out all test calls after you performed **run all cells**.
- __Make sure you fill in any place that says "#YOUR CODE HERE" or "YOUR ANSWER HERE".__
- __Comment the inserted code in details, otherwise you will not get the full marks even if the tests pass.__

__Note:__ Some code has been given in order to define the input and output.
Your code should be implemented between two comments "# YOUR CODE HERE" and "#YOUR CODE FINISHED ABOVE" ie. your code should replace the line "raise NotImplementedError". Please make sure that you do not miss any blocks, except for the example below:

```
    # YOUR CODE HERE
    raise NotImplementedError
    # YOUR CODE FINISHED ABOVE
```

In addition, you must use version 3 or greater of the IPython/Jupyter notebook for automarking to work properly. If you are not using version 3+, it is possible to accidentally delete cells that contain important metadata for automarking. To ensure that you have a recent enough version of the notebook, you can run the cell below to check:

In [1]:
import IPython
your_version = IPython.version_info[0]
print(your_version)
assert your_version >= 3, "Your version of IPython is too old, please update it."

7


This assignment requires __Python 3__ and its package *numpy, pandas* and *matplotlib*, so you need to make sure they are available from your Jupyter notebook.

In [2]:
from platform import python_version
print("python", python_version())

# for arrays
import numpy as np
print("numpy", np.version.version)

# for data frames (i.e tabular data that have row names and column names)
# unlike numpy arrays data frames support different data types for each column 
import pandas 
print("pandas", pandas.__version__)

# for random sequences
import random

# for measuring execution time
import timeit

# for plotting
import matplotlib 
print("matplotlib", matplotlib.__version__)

python 3.7.4
numpy 1.17.2
pandas 0.25.1
matplotlib 3.1.1


## Question 1 (3 points)

Implement the Needleman–Wunsch algorithm for global sequence alignment. This implementation should be split into two parts: 

- creating F matrix
- tracing back

__Note:__ _numpy_ is imported as _np_. Make sure the nucleotide/amino acid gap symbol is using hyphen "-".


### To do:
- Complete and comment the code for function *get_F_matrix()* (0.7 point)
- Complete and comment the code for function  *trace_back()* (1 point)
- Complete and comment the code for function *get_optimal_score()* and *get_optimal_score2()* (0.6 point)
- Capture the execution time of *get_optimal_score()* for different lengths of the input sequences. <br> Plot and comment the results. For this task you will have to complete and comment two functions. (0.7 point)

### i. F matrix
Complete the function *get_F_matrix()* below to produce the 2D $F$ matrix using _numpy_. The scoring system is determined by three parameters with default values: match=1.0, mismatch=-1.0, gap=-2.0, where _match/mismatch_ is equivalent to the score matrix and _gap_ is the linear gap penalty.
The sequence _A_ is always in the row header, and _B_ in column header.


In [3]:
def get_F_matrix(A, B, match=1.0, mismatch=-1.0, gap=-2.0):
    """
    Returns a global alignment between sequences A and B,
    gap is the linear gap penalty.
    F is the F matrix, where A is in row, B is in column.
    Strings AlignmentA and AlignmentB make up the alignment. 
    """
    
    # The dimension of the matrix
    rown = len(A) + 1
    coln = len(B) + 1

    # init 2D F matrix, shape=(len(A)+1, len(B)+1)
    F = np.zeros(shape=(rown, coln))
    
    # YOUR CODE HERE

    F[0 , :] = [gap * _ for _ in range(coln)]  # init first row of F
    F[: , 0] = [gap * _ for _ in range(rown)]  # init first col of F
    
    for i in range(1, rown):
        
        for j in range(1, coln):
            
            # If characters are the same, it's a match, otherwise, mismatch
            case1 = F[i-1,j-1] + match if (A[i-1] == B[j-1]) else F[i-1, j-1] + mismatch
            
            case2 = F[i, j-1] + gap  # Align with a gap in A seq
            
            case3 = F[i-1, j] + gap  # Align with a gap in B seq
            
            # Take the max of the three
            F[i, j] = max(case1, case2, case3)
    
    # YOUR CODE FINISHED ABOVE
    
    return F



In [4]:
"""AUTOMARKING F matrix calculation (simple test)"""
x = "GATTACA"
y = "GCATGCU"
mat = get_F_matrix(x,y,gap=-1)
print(mat)

assert np.array_equal(mat, 
                      np.reshape((0, -1, -2, -3, -4, -5, -6, -7,
                                  -1,  1,  0, -1, -2, -3, -4, -5,
                                  -2,  0,  0,  1,  0, -1, -2, -3,
                                  -3, -1, -1,  0,  2,  1,  0, -1,
                                  -4, -2, -2, -1,  1,  1,  0, -1,
                                  -5, -3, -3, -1,  0,  0,  0, -1,
                                  -6, -4, -2, -2, -1, -1,  1,  0,
                                  -7, -5, -3, -1, -2, -2,  0,  0), (8, 8)) )

[[ 0. -1. -2. -3. -4. -5. -6. -7.]
 [-1.  1.  0. -1. -2. -3. -4. -5.]
 [-2.  0.  0.  1.  0. -1. -2. -3.]
 [-3. -1. -1.  0.  2.  1.  0. -1.]
 [-4. -2. -2. -1.  1.  1.  0. -1.]
 [-5. -3. -3. -1.  0.  0.  0. -1.]
 [-6. -4. -2. -2. -1. -1.  1.  0.]
 [-7. -5. -3. -1. -2. -2.  0.  0.]]


In [5]:
"""AUTOMARKING F matrix calculation (hidden test)"""

'AUTOMARKING F matrix calculation (hidden test)'

MANUAL MARKING the coding style of the get_F_matrix() function.

### ii. Tracing back

Complete the function *trace_back()* to form the alignment given a scoring matrix $F$ created by *get_F_matrix()*, and other required parameters. 

If two or more scores are the same, select the high road during traceback.
For example, if there is the multiple choice to move either vertical or diagonal, then the high road approach will take the vertical move. If either diagonal or horizontal, then take diagonal. 

In [6]:
def trace_back(F, A, B, match=1.0, mismatch=-1.0, gap=-2.0):
    """
    Returns a global alignment between two sequences A and B.
    gap is the linear gap penalty
    F is the F matrix, where A is in row, B is in column.
    Strings AlignmentA and AlignmentB make up the alignment. 
    """
    # i indexing row, j indexing col in F
    i, j = len(A), len(B)
    if i == 0 or j == 0 or i >= np.shape(F)[0] or j >= np.shape(F)[1]:
        raise ValueError('Invalid position [%s, %s] !' % (i, j))

    # aligned sequence A
    AlignmentA = ""
    # aligned sequence B
    AlignmentB = ""
    
    # YOUR CODE HERE
    
    # While there still sequences left:
    while i > 0 or j > 0:

        # If move vertically is optimal, insert gap in B sequence.
        if F[i, j] == F[i - 1, j] + gap:
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = '-' + AlignmentB
            i -= 1  # Decrement i

        # If move diagonally is optimal, insert one char in both sequences.
        elif F[i, j] == F[i - 1, j - 1] + (match if (A[i - 1] == B[j - 1]) else mismatch):
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            i -= 1  # Decrement both i and j
            j -= 1
        
        # The only case left, if move horizontally is optimal, insert gap in A sequence.
        else:
            AlignmentA = '-' + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            j -= 1  # Decrement j
    
    # YOUR CODE FINISHED ABOVE
    
    # returns two strings
    return AlignmentA, AlignmentB
x = "AGGC"
y = "ATGT"
mat = get_F_matrix(x,y,match=1,mismatch=-1,gap=-1)
print(mat)
alignment = trace_back(mat, x, y,match=1,mismatch=-1, gap=-1)
print("y: ", alignment[1], "\nx: ", alignment[0])

[[ 0. -1. -2. -3. -4.]
 [-1.  1.  0. -1. -2.]
 [-2.  0.  0.  1.  0.]
 [-3. -1. -1.  1.  0.]
 [-4. -2. -2.  0.  0.]]
y:  ATGT 
x:  AGGC


In [7]:
"""AUTOMARKING global sequence alignment (simple test) """
x = "GATTACA"
y = "GCATGCU"
mat = get_F_matrix(x,y,gap=-1)
alignment = trace_back(mat, x, y, gap=-1)
print("y: ", alignment[1], "\nx: ", alignment[0])

assert alignment[1] == "GCATG-CU"
assert alignment[0] == "G-ATTACA"

y:  GCATG-CU 
x:  G-ATTACA


In [8]:
"""AUTOMARKING global sequence alignment (hidden test 1) """

'AUTOMARKING global sequence alignment (hidden test 1) '

In [9]:
"""AUTOMARKING global sequence alignment (hidden test 2)"""

'AUTOMARKING global sequence alignment (hidden test 2)'

MANUAL MARKING the coding style of the trace_back() function.

### iii. Calculating optimal score 
Complete the function *get_optimal_score()*,  a wrapper of the *get_F_matrix()* to return only the score of the optimal global alignment.

Complete the function *get_optimal_score2()*, that will calculate the score of the optimal global alignment using less memory space than the *get_optimal_score()*.
Comment the memory usage of both functions, i.e. memory storage as function of the length of input sequences.

In [10]:
def get_optimal_score(A, B, match=1.0, mismatch=-1.0, gap=-2.0):
    """
    Calculates and returns the score of the optimal global alignment of sequences A and B.
    gap is the linear gap penalty.
    """
    
    # YOUR CODE HERE
    
    # By definition, the bottom right element of the F matrix 
    # is the optimal score of A[:] and B[:]
    F = get_F_matrix(A, B, match, mismatch, gap)
    score = F[-1, -1]
    
    # YOUR CODE FINISHED ABOVE
    
    return score

def get_optimal_score2(A, B, match=1.0, mismatch=-1.0, gap=-2.0):
    """
    Calculates and returns the score of the optimal global alignment of sequences A and B.
    Uses memory efficient score calculation. 
    gap is the linear gap penalty.
    """

    # YOUR CODE HERE
    
    # Dimension of the matrix
    rown = len(A) + 1
    coln = len(B) + 1
    
    # Init F to be a 2 by coln matrix, and i will only keep 2 rows at all time to save memory
    F = np.zeros(shape=(2, coln))
    F[0 , :] = [gap * _ for _ in range(coln)]  # init first row of F
    F[1 , 0] = gap  # The first col of second row is just the gap value
    
    c = 1  # A counter for the F matrix 
    while c < rown:
        
        F[1,0] = c * gap  # First col of the second row is always a linear gap value
        
        for j in range(1, coln):
        
            # If characters are the same, it's a match, otherwise, mismatch
            case1 = F[0,j-1] + match if (A[c-1] == B[j-1]) else F[0, j-1] + mismatch
            
            case2 = F[1, j-1] + gap  # Align with a gap in A seq
            
            case3 = F[0, j] + gap  # Align with a gap in B seq
            
            # Set the current score to the max of three
            score = max(case1,case2, case3)
            F[1,j] = score
            
        # Old second row become the new first row
        F[0,:] = F[1,:]
        # Init new second row (from the second element) to be zero, the first will be set to gap value
        F[1,1:] = [0] * (coln - 1)

        c += 1  # Increment counter

   
    score = F[0, -1]  # The optimal score is the first row, last col of the final iteration of F matrix.
    
    # YOUR CODE FINISHED ABOVE
                
    return score

In [11]:
"""AUTOMARKING optimal score calculation (simple test)"""
x = "GATTACA"
y = "GCATGCU"

score_1 = get_optimal_score(x, y, gap=-1)
score_2 = get_optimal_score2(x, y, gap=-1)

assert score_1 == score_2
assert score_1 == 0.0
assert score_2 == 0.0

In [12]:
"""AUTOMARKING optimal score calculation (hidden test)"""


'AUTOMARKING optimal score calculation (hidden test)'

MANUAL MARKING the coding style of the get_optimal_score() and get_optimal_score2() functions

### iv. Empirically validate the time performance of *get_optimal_score()*

What is the time complexity of the *get_optimal_score()* function? <br>
Explain it briefly and show it empirically by running experiments with different pair of sequences of given length and capturing the execution time of the function. For this task is sufficient to align randomly generate sequences (DNA or protein) of the same length N. Complete function *random_sequence()* to generate random sequences of a given length.

Let the length of the sequence N be in the range 10 (parameter *min_N*) to 511 (parameter *max_N*) with step of 100 (parameter *step_N*). For each N, sample 5 pairs (parameter *pairs_N*) of randomly generated sequences of length N, and time the execution of the *get_optimal_score()* for each pair. Plot the results with the execution time (in seconds) on Y axis and the length of the input sequences on X axis. Label the plot properly, and comment the plot. This can take quite some time for large value of N, so for testing purposes use smaller ranges, i.e. the default parameters in the function signature bellow. Only when your function is free of errors run it for the requested setting.

__Hint:__
Use the functions *timeit.timeit(..., number=100)* or *timeit.repeat(..., number=100, repeat=5)* for recording the execution time, and *random.choice()* for random sampling. Please read the documentation of the corresponding Python modules to properly use them. Note, only basic plotting with *matplotlib* and tabular data manipulation with *pandas* is needed. 

In [13]:
import matplotlib.pyplot as plt
from functools import partial
%matplotlib inline

def random_sequence(alphabet='CGTA', N=5):
    """
    Generates a random sequence of length N using letters from alphabet, the set of possible letters.  
    Each letter has equal probability to be chosen at any position of the sequences
    """

    # YOUR CODE HERE
    
    sequence = ""  # init sequence
    
    while len(sequence) < N:
        
        # Generate random num in the range of (0, len(alphabet) - 1) as index, 
        # append the corresponding character to the seq.
        sequence += alphabet[random.randint(0,len(alphabet) - 1)]  

    # YOUR CODE FINISHED ABOVE
    
    # returns the random sequence as a string
    return sequence

def time_and_plot(alphabet='CGTA', pairs_N =5, min_N = 5, max_N = 21, step_N = 5,
                  match=1.0, mismatch=-1.0, gap=-2.0):
    """
    Empirical analysis of time performance of get_optimal_score()
    Calculates and save the times in a pandas.DataFrame
    Calls functions random_sequences() and get_optimal_score()
    """
    
    # We set the seed of the random number generator for output reproducibility.
    random.seed(12345)
    
    # YOUR CODE HERE
    
    N_list = [_ for _ in range(min_N, max_N + 1, step_N)]  # Init list of length N.

    seq_list = []  # Init random sequence list
    
    # For each N:
    
    for n in N_list:
        
        # pairs_N pairs of random sequence.
        i = 0
        while i < pairs_N:
            
            # Add one pair(tuple) of random sequence of length n to the list.
            seq_list.append(
                (random_sequence(N = n),
                 random_sequence(N = n))
            )
            i += 1
    
    #  Convert to np.array for easy processing
    array_list = np.array(seq_list)
    
    
    # Above is to get the random sequence
    ########################################################################
    # Below is to process the sequence and get the time data
    
    
    time_list = []  # Init time list
    
    #  For each length 
    for i in range(len(N_list)):
        
        j = 0  #  Counter for the sequence of the same length
        total_time = 0  #  This is the total time for the sequence of the same length
        
        
        #  Total of pairs_N pair of sequences of the same length
        while j < pairs_N:

            curr = array_list[i * pairs_N + j]  # Current pair of sequence
            
            #  Add the processing time of the current pair of sequences to the total for the sequence of this length
            total_time += timeit.timeit(partial(get_optimal_score,curr[0],curr[1],match,mismatch,gap), number = 100)
            
            j += 1  # Increment counter

        
        average_time = total_time / pairs_N  #  Get the average time of sequence of this length

        time_list.append(average_time)  # Add the time to the time list

    
    #  Create a pandas.DataFrame using the length of sequences (N_list) and time (time_list)
    time_df = pandas.DataFrame({"N": N_list,   # List of length of sequences
                                "Time": time_list})  # List of processing time of sequences of same length 
    
    
    
    ########################################################################
    # Below is to create plot using matplotlib.pyplot
    
    fig1, ax1 = plt.subplots()  # Init plot

    ax1.plot(time_df["N"], time_df["Time"], label="k")  # Graph

    ax1.set_xlabel("Length of input sequences")  # X Axie label
    ax1.set_ylabel("Execution time (seconds)")  # Y Axie label
    
    ax1.set_title("Time performance of get_optimal_score()")  # Title

    # YOUR CODE FINISHED ABOVE
    
    # data frame with two columns N, Time
    # Time is the average execution time for the optimal score of pair_N pairs of sequences of length N
    return time_df

In [14]:
""" Test the functions"""
""" IMPORTANT: Uncomment the commands for testing purposes. Before you submit the final version version of this notebook
    PLEASE COMMENT OUT the commands in this cell. We will evaluate the functions in the hidden test."""
# time_df = time_and_plot(pairs_N = 5, min_N = 5, max_N = 21, step_N = 5)           
# time_df = time_and_plot(pairs_N = 5, min_N = 10, max_N = 511, step_N = 100)
# print(time_df)

' IMPORTANT: Uncomment the commands for testing purposes. Before you submit the final version version of this notebook\n    PLEASE COMMENT OUT the commands in this cell. We will evaluate the functions in the hidden test.'

In the cell below, please answer the question about the time complexity of the *get_optimal_score()* function and explain the results obtained from executing time_and_plot(pairs_N =5, min_N = 10, max_N = 511, step_N = 100). You should hard-code the results from this function call (the data frame returned) in a table, so they are not overridden, when we autograde the code.

Table template:

| Row | Length | Average Time (sec) |
| :-- | :-- | :-- |
| 1 | N1 | T1 |
| 2 | N2 | T2 |
...


### Discussion of Results for Q1. iii) ###
For *get_optimal_score()*, we will keep the entire F matrix in memory at all time, thus it will take up much more memory space as the input sequences get longer. As for *get_optimal_score2()*, we only keep a record of 2 rows of the entire matrix because when we fill in the entry for one row, only the value in the upper row and the current row is relevant to our calculation. Therefore, it will only take $\frac{2}{rown}$ of the memory taken by *get_optimal_score()* in terms of the length of input sequences.

### Discussion of Results for Q1. iv) ###
| Row | Length | Average Time (sec) |
| :-- | :-- | :-- |
| 1 |  10  |  0.024351 |
| 2 | 110 |  2.243196 |
| 3 | 210 |  8.227377 |
| 4 | 310 | 17.892721 |
| 5 | 410 | 31.405304 |
| 6 | 510 | 50.222010 |

We can see from the graph and the corresponding table of the result from executing time_and_plot(pairs_N =5, min_N = 10, max_N = 511, step_N = 100) that there's a positive correlation between the two factor, as the length of the sequence increases by 100 each time, the average time to process the sequences increases, but not linearly proportional. 

In [15]:
"""AUTOMARKING time complexity (hidden test)"""

'AUTOMARKING time complexity (hidden test)'

MANUAL MARKING the coding style of the time_and_plot() and random_sequence() functions

## Question 2 (2 points)

Implement the Smith–Waterman algorithm for local sequence alignment. 

### To do:
- Complete and comment *local_alignment()* (2 points; 1 point for F matrix calculation and 1 point for the traceback)

Complete the function *local_alignment()* to produce a local sequence alignment. A score matrix $S$ is required. Remember to always take the high road during traceback. If there are multiple maximum scores, then always take the maximum score in the lowest and then rightmost position in $F$.

Please do not change the code to define score matrix S1 and S2

In [16]:
# define score matrix S1 and S2

nuc = ['A', 'G', 'C', 'T'] # Purines, Pyramidines
# match=1, mismatch=-1
match_mismatch = np.reshape((1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1), (4, 4))
# score matrix:
S1 = pandas.DataFrame(match_mismatch, index=nuc, columns=nuc)
print("S1 = \n", S1)
# 2 if a = b, 1 if a b is purine or a b is pyrimidine,
# -2 if a is a purine and b is a pyrimidine or vice versa.
pur_pyr = np.reshape((2, 1, -2, -2, 1, 2, -2, -2, -2, -2, 2, 1, -2, -2, 1, 2), (4, 4))
S2 = pandas.DataFrame(pur_pyr, index=nuc, columns=nuc)
print("\nS2 = \n", S2)

S1 = 
    A  G  C  T
A  1 -1 -1 -1
G -1  1 -1 -1
C -1 -1  1 -1
T -1 -1 -1  1

S2 = 
    A  G  C  T
A  2  1 -2 -2
G  1  2 -2 -2
C -2 -2  2  1
T -2 -2  1  2


In [17]:
def local_alignment(A, B, S, d=-2.0):    
    """
    Returns a local alignment between sequences A and B,
    S is the score matrix as pandas.DataFrame, and d is the penalty.
    F is the F matrix, where A is in row, B is in column.
    max_pos is the coordinate of the highest score max_score in F matrix.
    Strings AlignmentA and AlignmentB make up the alignment. 
    """
    
    # The dimension of the matrix
    rown = len(A) + 1
    coln = len(B) + 1

    # init 2D F matrix, shape=(len(A)+1, len(B)+1)
    F = np.zeros(shape=(rown, coln))
    
    # holds the highest score in F matrix
    # initialize to 0 
    max_score = 0
    # coordinates of the highest score in F matrix as a tuple, 
    # i.e. (row index, col index)
    max_pos = None  

    AlignmentA = ""
    AlignmentB = ""
    
    # YOUR CODE HERE
    
    ###############################################################################################
    ###  Get F matrix
    
    F[0 , :] = [0 for _ in range(coln)]  # init first row of F to 0
    F[: , 0] = [0 for _ in range(rown)]  # init first col of F to 0
    
    for i in range(1, rown):
        
        for j in range(1, coln):
            
            case1 = F[i-1,j-1] + S[A[i-1]][B[j-1]]  # Align using the scoring matrix S
            
            case2 = F[i, j-1] + d  # Align with a gap in A seq
            
            case3 = F[i-1, j] + d  # Align with a gap in B seq
            
            case4 = 0  # No alignment
            
            score = max(case1, case2, case3, case4)  # Get the max
            F[i, j] = score  # Set the entry of the F matrix to the max score
            
            # If current score is the global max, set max_score and max_pos to it
            if (score > max_score):
                
                max_score = score
                max_pos = (i, j)

    
    ###############################################################################################
    ###  Trace Back
    
    
    i, j = max_pos  # Set starting point to the max_pos

    # Keep loop until reach score 0:
    while F[i][j] != 0:

        # If move vertically is optimal, insert gap in B sequence.
        if F[i, j] == F[i - 1, j] + d:
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = '-' + AlignmentB
            i -= 1  # Decrement i

        # If move diagonally is optimal, insert one char in both sequences.
        elif F[i, j] == F[i - 1, j - 1] + S[A[i - 1]][B[j - 1]]:
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            i -= 1  # Decrement both i and j
            j -= 1
        
        # The only case left, if move horizontally is optimal, insert gap in A sequence.
        else:
            AlignmentA = '-' + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            j -= 1  # Decrement j
    
    # YOUR CODE FINISHED ABOVE
                                       
    return AlignmentA, AlignmentB, max_score, max_pos, F

x = "CCCCC"
y = "CCGGCCC"
AlignmentA, AlignmentB, max_score, max_pos, F = local_alignment(x, y, S1*10)
print("y: ", AlignmentB, "\nx: ", AlignmentA, "\n", F, 
      "\nmax score =", max_score, "\nmax score coordinate =", max_pos)

y:  CCGGCCC 
x:  CC--CCC 
 [[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10. 10.  8.  6. 10. 10. 10.]
 [ 0. 10. 20. 18. 16. 16. 20. 20.]
 [ 0. 10. 20. 18. 16. 26. 26. 30.]
 [ 0. 10. 20. 18. 16. 26. 36. 36.]
 [ 0. 10. 20. 18. 16. 26. 36. 46.]] 
max score = 46.0 
max score coordinate = (5, 7)


In [18]:
"""AUTOMARKING local sequence alignment (simple test)"""
x = "GGTTGACTA"
y = "TGTTACGG"
AlignmentA, AlignmentB, max_score, max_pos, F = local_alignment(x, y, S1*3)
print("y: ", AlignmentB, "\nx: ", AlignmentA, "\n", F, 
      "\nmax score =", max_score, "\nmax score coordinate =", max_pos)

assert AlignmentB == "GTT-AC"
assert AlignmentA == "GTTGAC"
assert max_score == 13.0

y:  GTT-AC 
x:  GTTGAC 
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  3.  1.  0.  0.  0.  3.  3.]
 [ 0.  0.  3.  1.  0.  0.  0.  3.  6.]
 [ 0.  3.  1.  6.  4.  2.  0.  1.  4.]
 [ 0.  3.  1.  4.  9.  7.  5.  3.  2.]
 [ 0.  1.  6.  4.  7.  6.  4.  8.  6.]
 [ 0.  0.  4.  3.  5. 10.  8.  6.  5.]
 [ 0.  0.  2.  1.  3.  8. 13. 11.  9.]
 [ 0.  3.  1.  5.  4.  6. 11. 10.  8.]
 [ 0.  1.  0.  3.  2.  7.  9.  8.  7.]] 
max score = 13.0 
max score coordinate = (7, 6)


In [19]:
"""AUTOMARKING local sequence alignment (hidden test 1)"""

'AUTOMARKING local sequence alignment (hidden test 1)'

In [20]:
"""AUTOMARKING local sequence alignment (hidden test 2)"""

'AUTOMARKING local sequence alignment (hidden test 2)'

In [21]:
"""AUTOMARKING local sequence alignment (hidden test 3)"""

'AUTOMARKING local sequence alignment (hidden test 3)'

MANUAL MARKING the coding style of the local_alignment() function.

### Question 3 (2.5 points)

Implement the overlap algorithm in the function *overlap()* An overlap alignment of two strings $v=v_1...v_n$ and $w = w_1...w_m$ returns the highest scoring global alignment between a $i$-suffix of $v$ ($v_i ..v_n$) and a $j$-prefix of $w$ ($w_1...w_j$), for all possible $i$ and $j$.

__Hint__: Modifying the Needleman–Wunsch algorithm. Essentially, you do not need to penalize for the the gaps at the beginning of the alignment and the end of a normal global alignment.  Think how to initialize the $F$ matrix, and where in the $F$ matrix you need to start the traceback. Furthermore, follow the same rule as above: if there are multiple choices with same score in the path, then always take the high road during traceback. Note that the alignment your function returns should include the starting gaps and end gaps. An example of two simple sequences and their alignment is available in the test bellow. 


### To do:
- Complete and comment the code for *readBLOSUM62()* (0.5 points)
- Complete and comment the code for *overlap()* (2 points)

__Note:__  For the first task, you will use the provided text file with the BLOSUM62 scoring matrix. You are not allowed to use ready functions from *pandas* to read the scoring matrix in one step. Instead you have to use the basic Python function *readline()* to parse the text file line by line and string manipulation to get the numbers. The file has no missing values in the columns/rows.

In [22]:
def readBLOSUM62():
    """
    Returns the BLOSUM62 scoring matrix in BLOSUM_df, a pandas.DataFrame 
    The scoring matrix is stored in a text file, provided to you as "blosum62.txt". 
    Comments in the file are marked with # sign, and these should be excluded.
    """
    # The fourth line in the file has 23 elements represnting 20 canonical and 3 ambiguous amino acids
    # So the scoring matrix is 23 x 23 
    amino_acids = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","B","Z","X"]
    col_names = amino_acids
    row_names = amino_acids
    # it is available with the Jupyter notebook  
    length = len(amino_acids)
    text_file = open("blosum62.txt", "r")
    
    # YOUR CODE HERE
    
    # Skip the first few lines starting with '#'
    line = text_file.readline()
    while line[0] == '#':
        line = text_file.readline()
    
    matrix = []  # Init matrix m
    
    # Keep looping until reaching EOF
    while line != '':

        line = text_file.readline().rstrip().split(' ')  # Trim '/n' and split into list by whitespace

        if line[0] == '': break  # Break if EOF

        processed_line = [int(x) for x in line[1:]]  # Leave the first char and convert the rest to integer
        matrix.append(processed_line)  # Add the processed line to the matrix

    BLOSUM_df = pandas.DataFrame(matrix, columns=amino_acids, index=amino_acids)  # Create DataFrame using col and row name from amino_acid
    
    text_file.close()  # Close file
    
    # YOUR CODE FINISHED ABOVE

    return BLOSUM_df

BLOSUM62 = readBLOSUM62()
print(list(BLOSUM62['A']))

[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0, -2, -1, 0]


In [23]:
"""AUTOMARKING readBLOSUM62 (hidden test)"""

'AUTOMARKING readBLOSUM62 (hidden test)'

MANUAL MARKING the commenting and coding style of the readBLOSUM62() function

In [24]:
def overlap(A,B,S,d=-2.0):
    
    """
    Returns an overlap alignment between sequences A and B,
    S is the score matrix, and d is the penalty.
    F is the F matrix, where A is in row, B is in column.
    String AlignmentA and AlignmentB are the alignment. 
    score is the optimal alignment score
    """
    # init 2D F matrix, shape=(len(A)+1, len(B)+1)
    F = np.zeros((len(A)+1,len(B)+1))
    
    AlignmentA = ""
    AlignmentB = ""
    
    # YOUR CODE HERE
    
    ###############################################################################################
    ### Get F matrix
    
    
    # Dimension of F matrix
    rown = len(A) + 1
    coln = len(B) + 1
    
    # Max score and position
    max_score = 0
    max_pos = None  
    
    # End_pos where the alignment ends
    end_pos = None

    F[0 , :] = [0 for _ in range(coln)]  # init first row of F to 0
    F[: , 0] = [0 for _ in range(rown)]  # init first col of F to 0
    
    for i in range(1, rown):
        
        for j in range(1, coln):
            
            # If characters are the same, it's a match, otherwise, mismatch
            case1 = F[i-1,j-1] + S[A[i-1]][B[j-1]]
            
            case2 = F[i, j-1] + d  # Align with a gap in A seq
            
            case3 = F[i-1, j] + d  # Align with a gap in B seq
            
            # Take the max of the three
            score = max(case1, case2, case3)
            F[i, j] = score
            
            # Find the maximum only on the last row of column
            if i == (rown - 1) or j == (coln - 1): 
                
                if (score > max_score):
                    max_score = score
                    max_pos = (i, j)
            
    ###############################################################################################
    ###  Trace Back
    
    i, j = max_pos  # Set starting point to the max_pos

    # Keep loop until reach score 0:
    while F[i][j] != 0:

        # If move vertically is optimal, insert gap in B sequence.
        if F[i, j] == F[i - 1, j] + d:
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = '-' + AlignmentB
            i -= 1  # Decrement i

        # If move diagonally is optimal, insert one char in both sequences.
        elif F[i, j] == F[i - 1, j - 1] + S[A[i - 1]][B[j - 1]]:
            AlignmentA = A[i - 1] + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            i -= 1  # Decrement both i and j
            j -= 1
        
        # The only case left, if move horizontally is optimal, insert gap in A sequence.
        else:
            AlignmentA = '-' + AlignmentA
            AlignmentB = B[j - 1] + AlignmentB
            j -= 1  # Decrement j
        
        # When the alignment end, set the end_pos
        if F[i][j] == 0:
            end_pos = (i, j)
    
    # B seq has some char left
    if end_pos[0] == 0:
        
        # Adding gap      
        AlignmentA = '-' * end_pos[1] + AlignmentA  # Add leading gap to alignment A
        AlignmentB = AlignmentB + '-' * (rown - max_pos[0] - 1)  # Add trailing  gaps to alignment B

        # Adding sequence
        AlignmentA = AlignmentA + A[max_pos[0]:]  # Add trailing sequence to alignment A
        AlignmentB = B[:end_pos[1]] + AlignmentB  # Add leading sequence to alignment B
    
    # A seq has some char left
    elif end_pos[1] == 0:
        
        # Adding gap      
        AlignmentB = '-' * end_pos[0] + AlignmentB  # Add leading gap to alignment B
        AlignmentA = AlignmentA + '-' * (coln - max_pos[1] - 1)  # Add trailing  gaps to alignment A

        # Adding sequence
        AlignmentB = AlignmentB + B[max_pos[1]:]  # Add trailing sequence to alignment B
        AlignmentA = A[:end_pos[0]] + AlignmentA  # Add leading sequence to alignment B
    
    
    # YOUR CODE FINISHED ABOVE

    return AlignmentA, AlignmentB, max_score, F

x = 'CCCCC'
y = 'GGGGGGG'

AlignmentA,AlignmentB,score,F = overlap(x, y, BLOSUM62, -8)
print(AlignmentB)
print(AlignmentA)
print(score)
print(F)


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
"""AUTOMARKING overlap alignment (simple test)"""
x = 'PAWHEAE'
y = 'HEAGAWGHEE'

AlignmentA,AlignmentB,score,F = overlap(x, y, BLOSUM62, -8)
print(AlignmentB)
print(AlignmentA)
print(score)
print(F)

assert AlignmentB == "HEAGAWGHEE-"
assert AlignmentA == "---PAW-HEAE"
assert score == 17.0

In [ ]:
"""AUTOMARKING overlap alignment using BLOSUM62 (hidden test 1)"""

In [ ]:
"""AUTOMARKING overlap alignment using BLOSUM62 (hidden test 2)"""

In [ ]:
"""AUTOMARKING overlap alignment using BLOSUM62 (hidden test 3)"""

MANUAL MARKING the commenting and coding style of the overlap() function